# Random Useless Facts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iragca/pit-predictive-analysis/blob/master/notebooks/1.0-random-useless-facts.ipynb)

## Executive Summary



## Table of Contents

```bash
.
├── Title Section
│       ├── Executive Summary
│       ├── Table of Contents                   #  ⭐ You are here
│       └── Setup                               #  Libraries and dataset imports
├── Introduction
│       ├── Problem Statement
└── Methodology
```


## Setup


In [ ]:
try:
    import google.colab  # type: ignore
    IS_COLAB_ENVIRONMENT = True
except ImportError:
    IS_COLAB_ENVIRONMENT = False

if IS_COLAB_ENVIRONMENT:
    from google.colab import userdata # type: ignore
    %pip install iragca
    %pip install pydantic
    %pip install supabase

In [11]:
from enum import Enum
import os
from urllib.parse import urlencode, urljoin

import dotenv
import httpx
from pydantic import BaseModel
from supabase import Client, create_client

dotenv.load_dotenv()

True

# Introduction

## Problem Statement

What are the most common topics being observed despite the mundaneness of the subject matter?

# Methodology

## API Wrapper

In [12]:
class Language(str, Enum):
    """
    Enumeration of supported languages for the Random Useless Facts API.

    Each enum member corresponds to the language code expected by the
    remote API.
    """
    ENGLISH = "en"
    SPANISH = "es"
    FRENCH = "fr"
    GERMAN = "de"
    ITALIAN = "it"

class Fact(BaseModel):
    """
    Data model representing a response returned by the Random Useless Facts API.

    Parameters
    ----------
    id : str
        Unique identifier of the fact.
    text : str
        The textual content of the fact.
    source : str
        Origin or reference for the fact.
    source_url : str
        URL pointing to the source.
    language : Language
        Language in which the fact is written.
    permalink : str
        Permanent URL for accessing the fact.

    Notes
    -----
    This model is parsed from JSON using Pydantic and ensures type validation
    of the incoming API response.
    """
    id: str
    text: str
    source: str
    source_url: str
    language: Language
    permalink: str


class RandomUselessFactAPI:
    """
    Client for interacting with the Random Useless Facts API.

    Parameters
    ----------
    base_url : str, optional
        Base URL of the API. Defaults to ``"https://uselessfacts.jsph.pl/"``.

    Notes
    -----
    This class exposes convenience methods to construct API URLs and retrieve
    random facts in various languages. Requests are executed using ``httpx``.
    """
    def __init__(self, base_url: str ="https://uselessfacts.jsph.pl/") -> None:
        self.base_url = base_url

    def get_random_fact(self, language: Language = Language.ENGLISH) -> Fact:
        """
        Retrieve a random useless fact.

        Parameters
        ----------
        language : Language, optional
            Language in which the fact should be returned. Defaults to
            ``Language.ENGLISH``.

        Returns
        -------
        Response
            Parsed API response containing the random fact.

        Raises
        ------
        httpx.HTTPError
            If the request fails or returns an error status code.

        Examples
        --------
        >>> api = RandomUselessFactAPI()
        >>> fact = api.get_random_fact()
        >>> fact.text
        'Banging your head against a wall for one hour burns 150 calories.'
        """
        url = self._build_random_fact_url(language)
        response = httpx.get(url)
        response.raise_for_status()
        data = response.json()

        return Fact(**data)
    
    def _build_random_fact_url(self, language: Language = Language.ENGLISH) -> str:
        """
        Construct the request URL for fetching a random fact.

        Parameters
        ----------
        language : Language, optional
            Language code to include as a query parameter. Defaults to
            ``Language.ENGLISH``.

        Returns
        -------
        str
            Fully constructed URL pointing to the random fact endpoint.

        Notes
        -----
        This method is internal and is not intended to be called directly by
        users. It performs safe URL joining and query parameter encoding.
        """
        ENDPOINT = "api/v2/facts/random"
        path = urljoin(self.base_url, ENDPOINT)
        params = urlencode({"language": language.value})
        
        if params:
            return f"{path}?{params}"
        return path
    

In [13]:
api = RandomUselessFactAPI()

In [14]:
fact: Fact = api.get_random_fact(language=Language.ENGLISH)
print(fact.text)

Former basketball superstar Michael Jordan is the most recognized face in the world, more than the pope himself.


In [15]:
fact

Fact(id='ec36d7884a85ddbf09c49da9fd299660', text='Former basketball superstar Michael Jordan is the most recognized face in the world, more than the pope himself.', source='djtech.net', source_url='http://www.djtech.net/humor/useless_facts.htm', language=<Language.ENGLISH: 'en'>, permalink='https://uselessfacts.jsph.pl/api/v2/facts/ec36d7884a85ddbf09c49da9fd299660')

In [ ]:
from supabase import create_client, Client

def load_env():
    if IS_COLAB_ENVIRONMENT:
        url: str = userdata.get("SUPABASE_URL")
        key: str = userdata.get("SUPABASE_KEY")
    else:
        url: str = os.environ.get("SUPABASE_URL", "")
        key: str = os.environ.get("SUPABASE_KEY", "")
    return url, key

url, key = load_env()

if key and url:
    supabase: Client = create_client(url, key)
else:
    raise ValueError("SUPABASE_URL and SUPABASE_KEY must be set in environment variables.")

In [18]:
response = (
    supabase.table("facts")
    .insert(api.get_random_fact().model_dump())
    .execute()
)
response

APIResponse(data=[{'id': '8e9d9964ba9f8a7438a34ade2d948fae', 'text': "In Japan, watermelons are squared. It's easier to stack them that way.", 'source': 'djtech.net', 'source_url': 'https://www.djtech.net/humor/shorty_useless_facts.htm', 'language': 'en', 'permalink': 'https://uselessfacts.jsph.pl/api/v2/facts/8e9d9964ba9f8a7438a34ade2d948fae', 'created_at': '2025-12-03T08:44:14.517774+00:00'}], count=None)